### Is this just magic?  What is Numba doing to make code run quickly?

Short answer: Yes, it's magic.

Long answer: No, it's not magic, but it's a little complicated.  Here's a broad overview.

### Python bytecode

What actually happens when you execute a function written in Python?  It gets parsed and executed by CPython.  
Let's take a trivial example and see what's going on under the hood.

In [1]:
def add(a, b):
    return a + b

Import the Python disassembler

In [2]:
import dis

In [3]:
dis.dis(add)

  2           0 LOAD_FAST                0 (a)
              3 LOAD_FAST                1 (b)
              6 BINARY_ADD
              7 RETURN_VALUE


Makes sense.  Load `a` from the stack, load `b` from the stack, add them together and then return.  

In [22]:
from numba import jit

In [6]:
@jit
def add(a,b):
    return a + b

In [7]:
dis.dis(add)

  3           0 LOAD_FAST                0 (a)
              3 LOAD_FAST                1 (b)
              6 BINARY_ADD
              7 RETURN_VALUE


Huh?  It's doing nothing?  

In [8]:
add(1, 1)

2

In [12]:
dis.dis(add)

  3           0 LOAD_FAST                0 (a)
              3 LOAD_FAST                1 (b)
              6 BINARY_ADD
              7 RETURN_VALUE


In [11]:
add.inspect_types()

add (int64, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-6-a2aaf27e3e67>
# --- LINE 1 --- 

@njit

# --- LINE 2 --- 

def add(a,b):

    # --- LINE 3 --- 
    # label 0
    #   a = arg(0, name=a)  :: int64
    #   b = arg(1, name=b)  :: int64
    #   $0.3 = a + b  :: int64
    #   del b
    #   del a
    #   $0.4 = cast(value=$0.3)  :: int64
    #   del $0.3
    #   return $0.4

    return a + b




In [13]:
def add_object(a, b):
    return a.x + b.x

In [15]:
class MyInt(object):
    def __init__(self, x):
        self.x = x

In [19]:
a = MyInt(5)
b = MyInt(6)

In [20]:
add_object(a, b)

11

In [21]:
dis.dis(add_object)

  2           0 LOAD_FAST                0 (a)
              3 LOAD_ATTR                0 (x)
              6 LOAD_FAST                1 (b)
              9 LOAD_ATTR                0 (x)
             12 BINARY_ADD
             13 RETURN_VALUE


In [23]:
add_object_jit = jit(add_object)

In [24]:
add_object_jit(a, b)

11

In [25]:
add_object_jit.inspect_types()

add_object (pyobject, pyobject)
--------------------------------------------------------------------------------
# File: <ipython-input-13-e351116a80fa>
# --- LINE 1 --- 

def add_object(a, b):

    # --- LINE 2 --- 
    # label 0
    #   a = arg(0, name=a)  :: pyobject
    #   b = arg(1, name=b)  :: pyobject
    #   $0.2 = getattr(value=a, attr=x)  :: pyobject
    #   del a
    #   $0.4 = getattr(value=b, attr=x)  :: pyobject
    #   del b
    #   $0.5 = $0.2 + $0.4  :: pyobject
    #   del $0.4
    #   del $0.2
    #   $0.6 = cast(value=$0.5)  :: pyobject
    #   del $0.5
    #   return $0.6

    return a.x + b.x


